In [1]:
import sys
def progress(count, total, status=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))
    
    #old: percents = round(100.0 * count / float(total), 1)
    percents = round(100.1 * count / float(total), 1)
    bar = '#' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', status))
    sys.stdout.flush()  # As suggested by Rom Ruben (see: http://stackoverflow.com/questions/3173320/text-progress-bar-in-the-console/27871113#comment50529068_27871113)

In [2]:
## the Times XML parser p1
import csv
import os
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET


#defines the file's directory for the xml files
path = 'E:/storage-riddles-ra/agony-columns/corpora/1860'

def getMainAttribute(tree, writer, minOCR=40.0):

    #get root of xml file
    root = tree.getroot()
    
    #issue number
    # print(root.tag + root.attrib['ID'])
    issueID = root.tag+root.attrib['ID']

    #get date
    full_date = root.find('.//composed').text
    # print(date)

    #get width and height of the paper
    width = root.find('.//pageid').attrib['width']
    height = root.find('.//pageid').attrib['height']

    #Articles is defined as a list (we get all the articles in the paper)
    Articles = root.findall(".//article[@type='Article']")

    #loop through the articles
    for Article in Articles:
        #check for ocr scores
        ocrScore = float(Article.find(".//ocr").text)    
        if ocrScore < minOCR:
            break
        pageNumber = Article.find(".//pi[@pgref='1']")
           #breaks loop if we are not on the first page
        if pageNumber is None:
            break

        title = Article.find(".//ti").text
        # print(title)

        TextInArticle = Article.find(".//text")
        #get <text.cr>
        TextInput = ''
        for TextContainer in TextInArticle:
            #get <pg> and <p>
            for Paragraph in TextContainer:
                if Paragraph.tag == "pg":
                    positionValues = [int(lbtr) for lbtr in Paragraph.attrib['pos'].split(',')]
                    #if Paragraph.attrib['clipref'] != "2":
                    #    break

                #subparagraph gives us the wd positions
                
                for SubParagraph in Paragraph:
                    
                                #[0] = left
                                #[1] = bottom
                                #[2] = right
                                #[3] = top
                                #in case it's needed
                    subPositionValues = [int(lbtr) for lbtr in SubParagraph.attrib['pos'].split(',')]
                    if(int(subPositionValues[2]) <= (int(width)/6)+5):
                        # print("1st BREAK")
                        break    
                    elif(int(subPositionValues[2]) >= (3*(int(width)/6))+5):
                        # print("2nd BREAK")
                        break
                    else:
                        # print("3rd BREAK")
                        # print(SubParagraph.text)
                        TextInput += SubParagraph.text + ' '
                    #print(SubParagraph.text, end=' ')    
                    #print(positionValues[0],positionValues[1],positionValues[2],positionValues[3])
                #print(TextInput)
        writer.writerow([issueID, full_date, title, ocrScore, TextInput])


In [6]:
def write_newspaper_XMLs_to_df(name_of_csv='testrun', path_to_XML_sheets='E:/storage-riddles-ra/agony-columns/corpora/1860', minOCR=40.0):
    with open(name_of_csv+'.csv', 'w', newline='', encoding="utf8", errors='ignore') as csvFile:
        i = 0
        writer = csv.writer(csvFile) 
        writer.writerow(["issueID", "full_date", "title", "ocrScore", "text"])
        for files in os.listdir(path_to_XML_sheets):
            # progress(i, len(os.listdir(path)))
            if not files.endswith('.xml'):
                continue
            fullname = os.path.join(path_to_XML_sheets, files)
            tree  = ET.parse(fullname)
            getMainAttribute(tree, writer, minOCR)
            i += 1
            progress(i, len(os.listdir(path)))
    #group duplicate rows
    df = pd.read_csv(r'testrun.csv', delimiter=',')
    # new: df = pd.read_csv('testrun.csv', delimiter=',')
    #drop nan values
    df.replace(['None', 'nan'], np.nan, inplace=True)
    df.dropna(how='any', inplace=True)
    return df

def give_me_just_the_year_from_date(df, date_column='full_date'):
    for index, row in df.iterrows():
        full_date = row[date_column]
        full_date_split = full_date.split()
        df.at[index, 'text_year'] = full_date_split[2] 
    return df
#reformat for a new output csv
# out = df.astype(str).groupby(['date', 'issueID', 'title']).agg(', '.join)
# print(out)
# out.to_csv('formatted_output.csv')
# out.head(-20)

In [7]:
df = write_newspaper_XMLs_to_df()
df = give_me_just_the_year_from_date(df)
df.head()

,issueID,full_date,title,ocrScore,text,text_year
20,issueN0023341,"April 14, 1860",Deaths,56.37,"DEATHS (continued). On the 1lth inst., at 12, ...",1860
21,issueN0023341,"April 14, 1860","W.T.-Pray ANSWER the last LETTER, which",43.34,"WXT-PraY ANSWER the last LETTER, which stated ...",1860
31,issueN0023345,"April 19, 1860","MONUMENTS, Tombs, Chimneypieces, & c.-The",42.51,G. ' BS.-RETIUEN at once. Help will be obtaine...,1860
40,issueN0023350,"April 25, 1860",THE NECROPOLIS COMPANY undertake the,40.40,"L .-Y u remember your promiso of aid, if I re-...",1860
44,issueN0023351,"April 26, 1860",West-End Economic Funeral Com-,44.51,IF J. M. will COMMUNICATE with his friends imn...,1860


In [5]:
# new_df.to_csv("test_each_AC_same_row.csv")